In [2]:
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt

In [3]:
# Set the column width to 1000
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [4]:
base = Path("/Users/larsankile/code/surveillance-metric-2/data_original")

In [5]:
pools = pl.read_parquet(base / "factory/*.parquet")

pools.columns

['block_timestamp',
 'block_number',
 'transaction_hash',
 'log_index',
 'token0',
 'token1',
 'fee',
 'tickSpacing',
 'pool']

In [6]:
tokens = pl.read_parquet(base / "uniswap-v3-tokens.parquet")

tokens.columns

['address', 'symbol', 'name', 'decimals']

In [7]:
# Prepare token_info_df by selecting only the columns we need and transforming address to lowercase
token_info = tokens.select(
    pl.col("address").str.to_lowercase().alias("address"),
    pl.col("symbol"),
    pl.col("decimals"),
)

# Join for token0 information, transforming token0 to lowercase
result = pools.join(
    token_info,
    left_on=pl.col("token0").str.to_lowercase(),
    right_on="address",
    how="left",
).rename({"symbol": "symbol0", "decimals": "decimals0"})

# Join for token1 information, transforming token1 to lowercase
result = result.join(
    token_info,
    left_on=pl.col("token1").str.to_lowercase(),
    right_on="address",
    how="left",
).rename({"symbol": "symbol1", "decimals": "decimals1"})

# Exclude the address columns
result = result.select(pl.all().exclude(columns=["address", "address_right"]))

result

block_timestamp,block_number,transaction_hash,log_index,token0,token1,fee,tickSpacing,pool,symbol0,decimals0,symbol1,decimals1
"datetime[ns, UTC]",i64,str,i64,str,str,str,str,str,str,f64,str,f64
2023-12-10 23:33:23 UTC,18759113,"""0xe2699a839b73f5b01e3730d0fa62b4dcbb5c355cdd62fc0fdcb6c42841fc3a51""",0,"""0x882db217dbc4b7c52a79a95a02b4e78d408d56d2""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""100""","""1""","""0xfbd78d4608d349a321fc2d676d281e24ed1c0896""",null,null,"""WETH""",18.0
2023-08-28 00:01:11 UTC,18009432,"""0x719eb90c947666f108ea898f14ec44d0265db56143c9f1b550c4a467a76323ef""",0,"""0x42141cd6113355d87793ca8925331e23c5da2d9d""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""100""","""1""","""0xa6e9eb44afec77cece0baa5c65e2107b61341f5b""",null,null,"""WETH""",18.0
2023-03-15 15:53:35 UTC,16834308,"""0xbc53dc8c28ea33549676f117fde050c46c72226817a27caa82b9159561853500""",0,"""0x8d96b4ab6c741a4c8679ae323a100d74f085ba8f""","""0xdac17f958d2ee523a2206206994597c13d831ec7""","""100""","""1""","""0x5c2a6a370e63b4de95650edc565ad0fbab5312ae""","""BZR""",18.0,"""USDT""",6.0
2023-10-30 21:15:59 UTC,18465593,"""0x6f82e811ac19059adbc4f1cca90820bfbfb8309db1b279c116cab9abb59f7dba""",0,"""0x9d3351fc4fdc93c7efd353ac97c93b6df87cb443""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""100""","""1""","""0x73895b1666f0e116f74b6300094d53a479e499bf""",null,null,"""WETH""",18.0
2023-11-08 20:30:35 UTC,18529635,"""0x6e7cee2611cd01be1524da1cce4881d5401b286c5fc0f35a21a493db7e58c8e4""",0,"""0x441de36e12d514bdd9203dc7c4f16fa2aba86571""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""100""","""1""","""0x0d56e909e585913023d1102f70088d7deba5411b""",null,null,"""WETH""",18.0
…,…,…,…,…,…,…,…,…,…,…,…,…
2022-08-10 02:16:11 UTC,15311689,"""0xd571c1e535c48aae9b3482261c52b2e0ed9eef42ec92494d59c177c743baafeb""",511,"""0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48""","""0xc55126051b22ebb829d00368f4b12bde432de5da""","""10000""","""200""","""0x491a8c1413aec9eb55ee59b4ac956ba5814a22aa""","""USDC""",6.0,null,null
2022-07-28 02:35:13 UTC,15228330,"""0x9c3e57be88b068e1f5603c2fa1dc558c5b3944099b601fb50dcc3a5060fe4e75""",511,"""0x097b9959bfcf715513ecd30c4b1fb469ef32b743""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""10000""","""200""","""0x453bc17c75fe452718d0f774bbc2b8c1d6684b70""",null,null,"""WETH""",18.0
2023-10-18 02:27:59 UTC,18374262,"""0xc310193244fc55476bc41fc52feafebd5718117d1ddfaf35bd19a903bc3c6294""",767,"""0x3be49de62c30cb373adf3adf6142489cfcbbf499""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""3000""","""60""","""0x5a48eb847972f84fe8f91a6fa1e6a27ad982e281""",null,null,"""WETH""",18.0


In [13]:
filtered_result = result.filter(
    pl.col("decimals0").is_not_null() & pl.col("decimals1").is_not_null()
)

filtered_result

chain_name,block_timestamp,block_number,transaction_hash,log_index,token0,token1,fee,tickSpacing,pool,symbol0,decimals0,symbol1,decimals1
str,"datetime[μs, UTC]",i64,str,i64,str,str,f64,str,str,str,f64,str,f64
"""ethereum""",2023-05-25 23:05:47 UTC,17339307,"""0x2f31841a5174d4470bd375358327473d735ec0be94718924fd39643f81b5c62a""",198,"""0x6cf4cf8f827b350c474c3862dbf13298977ee42d""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x7bf4eaced115b261e02737bd656571e4bc335549""","""REV""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 22:59:59 UTC,17339278,"""0x1334b6ea3d67bf7d0913764eb739b22fb4d7231f790b635ed7a6acce2dbc61ab""",299,"""0x40063963486c17be51743a18c016bfbd07e7ca15""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",3000.0,"""60""","""0x5081c559e31403b7e77532a5c3a2099d3da5e672""","""RBE""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 09:01:47 UTC,17335142,"""0x448fb8123f1d14109d2cd042332ab6e7596315fb02f07c14e39f82157b7d07bc""",264,"""0x0a4024d7ff45df2f9488649af70491d6beee6e80""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",3000.0,"""60""","""0x28f354bbb13f7d6738424cd3c092607c8f907999""","""MRW""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 15:45:59 UTC,17337135,"""0x39cda277528951896de1307c245991b5abb9fdb4672af7b07700d5aa06f3e1f0""",222,"""0x320dc0b6aafeb9fa7106799c9bbea3f02dde1f56""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x454743fb44e3d9fb1e21cc4b74e462e601140a49""","""CENSOR""",18.0,"""WETH""",18.0
"""ethereum""",2023-05-25 07:43:11 UTC,17334755,"""0xba1b2b6941763e2c2d32065fec3a50a42e2628a2c138215354987786995dfad9""",173,"""0x3cbbd2511139fd37497a5d01321b6aaf52c2b42f""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x2186bb4bad31902290a64e30912c3d8360a7ecbc""","""SQUID""",18.0,"""WETH""",18.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ethereum""",2024-07-01 23:09:23 UTC,20214859,"""0xaae81e0efcfec01262024e2e7b070c1183ccba27672b1a5970d7804f42b7a23c""",0,"""0x0abdace70d3790235af448c88547603b945604ea""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""",10000.0,"""200""","""0x74f3710792b88bce2c80698efc8a7829ad37d777""","""DNT""",18.0,"""WETH""",18.0
"""ethereum""",2024-07-02 17:28:35 UTC,20220320,"""0x2995d0178413cb29f91fa464aae4d7e8f4aa5524de9e5a932ef92a4adb8afe44""",394,"""0x6ca9d0b650c7a4b3871ea898b9f1a7fd27fb1d58""","""0xdac17f958d2ee523a2206206994597c13d831ec7""",100.0,"""1""","""0xe1ab3ac0495f991d2295b760418e255f4343571a""","""USD""",18.0,"""USDT""",6.0
"""ethereum""",2024-07-02 02:29:47 UTC,20215859,"""0xfa0326f6103cd8eb15e375a86f9afec5385728a3592f3424a52051f38fdaef66""",522,"""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xf230b790e05390fc8295f4d3f60332c93bed42e2""",10000.0,"""200""","""0x6792ff48f3164030f09f21cc5894fd5644418c30""","""WETH""",18.0,"""TRX""",6.0


In [14]:
# Save the result to a parquet file
filtered_result.write_parquet(base / "pool_token_info.parquet")